In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras.utils import to_categorical

In [2]:
train = keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/sleeve/Sleeve/train',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 16,
    image_size = (512,512)
)

Found 620 files belonging to 2 classes.


In [3]:
test = keras.utils.image_dataset_from_directory(
    directory = '/kaggle/input/sleeve/Sleeve/test',
    labels = 'inferred',
    label_mode = 'int',
    batch_size = 16,
    image_size = (512,512)
)

Found 272 files belonging to 2 classes.


In [4]:
def normalize(image, label):
  image = tf.cast(image/255 , tf.float32)
  return image, label

train = train.map(normalize)
test = test.map(normalize)

In [14]:
from keras.applications import VGG16
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import Adam

# Load the pre-trained VGG16 model without the top (fully connected layers)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(512, 512, 3))

# Freeze the weights of the pre-trained layers
for layer in base_model.layers:
    layer.trainable = False

# Add your own classification layers on top of the pre-trained model
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(64, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

# Create the fine-tuned model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, 512, 512, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 512, 512, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 512, 512, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 256, 256, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 256, 256, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 256, 256, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 128, 128, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 128, 128, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 128, 128, 256)  │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 128, 128, 256)  │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 64, 64, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 64, 64, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 64, 64, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 64, 64, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 32, 32, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 32, 32, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 16, 16, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 131072)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 512)            │    67,109,376 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_8 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_9 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 128)            │        32,89

 Total params: 81,996,609 (312.79 MB)

 Trainable params: 67,281,921 (256.66 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [15]:
model.fit(train,epochs=20,validation_data = test)

Epoch 1/20


W0000 00:00:1714025882.071874      93 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


38/39 ━━━━━━━━━━━━━━━━━━━━ 0s 342ms/step - accuracy: 0.5131 - loss: 5.2906

W0000 00:00:1714025900.623790      94 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


39/39 ━━━━━━━━━━━━━━━━━━━━ 0s 488ms/step - accuracy: 0.5130 - loss: 5.2784

W0000 00:00:1714025902.140198      95 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


39/39 ━━━━━━━━━━━━━━━━━━━━ 33s 659ms/step - accuracy: 0.5128 - loss: 5.2668 - val_accuracy: 0.5882 - val_loss: 0.7050
Epoch 2/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 19s 494ms/step - accuracy: 0.5234 - loss: 2.4499 - val_accuracy: 0.5257 - val_loss: 0.7823
Epoch 3/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 20s 493ms/step - accuracy: 0.5337 - loss: 1.2427 - val_accuracy: 0.5294 - val_loss: 0.7067
Epoch 4/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 19s 493ms/step - accuracy: 0.5674 - loss: 0.8046 - val_accuracy: 0.5699 - val_loss: 0.6876
Epoch 5/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 19s 496ms/step - accuracy: 0.5746 - loss: 0.7454 - val_accuracy: 0.6434 - val_loss: 0.6237
Epoch 6/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 20s 499ms/step - accuracy: 0.6754 - loss: 0.6022 - val_accuracy: 0.7096 - val_loss: 0.5619
Epoch 7/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 19s 497ms/step - accuracy: 0.7757 - loss: 0.4898 - val_accuracy: 0.7169 - val_loss: 0.5602
Epoch 8/20
39/39 ━━━━━━━━━━━━━━━━━━━━ 20s 499ms/step - accuracy: 0.8241 - loss: 0.3820 - val_accuracy: 0.783

In [16]:
model.save('Sleeve_Classifier.h5')